<a href="https://colab.research.google.com/github/adautofbn/ri_labs/blob/master/lab05/expansao_consulta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [193]:
import pandas as pd
import numpy as np
import nltk
import re
import collections
import heapq
import time
import bisect
import math
import random
from nltk.tokenize import RegexpTokenizer

nltk.download('stopwords')
result = pd.read_csv('https://raw.githubusercontent.com/adautofbn/ri_lab_01/master/output/results.csv') # Resultados adquiridos do site brasil-247

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
tknz = RegexpTokenizer(r'([A-Za-zÁáÉéÍíÓóÚúÃãÕõÇçÂâÊê]{3,27})')
stopwords = nltk.corpus.stopwords.words('portuguese') 
ranking = {}
n = 0

for text in result.text:
  words = [word for word in tknz.tokenize(text.lower())
           if not bool(re.search(r'\d', word))
           and word not in stopwords and len(word) >= 3]  
  n += 1
  for t in words:
    if t not in ranking.keys():
      ranking[t] = []
    ranking[t].append(n)
    
for elem in ranking.items():
  d = dict(collections.Counter(elem[1]))
  ranking[elem[0]] = list(d.items())

In [0]:
queries = ["bolsonaro", "educação", "ministério", "brasil", "economia"]

### 1. Calcule as top-10 palavras mais associadas a cada uma dessas 5 palavras de acordo com as 4 métricas que vimos na aula. Você deve produzir uma tabela similar à tabela 6.3 do capítulo 6 do livro texto (pág. 204). Qual métrica você acha que obteve os melhores resultados? Por que?

In [0]:
def n_docs_word(word):
  return len(ranking[word])

def n_docs_both(a, b):
  docs_a = [doc for doc,score in ranking[a]]
  docs_b = [doc for doc,score in ranking[b]]
  n = 0
  
  for doc in docs_a:
    if doc in docs_b:
      n += 1
      
  return n
  
def n_docs():
  return result.text.count()

In [0]:
def calc_mim(a, b):
  n_a = n_docs_word(a)
  n_b = n_docs_word(b)
  n_ab = n_docs_both(a,b)
  mim = n_ab / (n_a * n_b)
  return (mim,b)
  
def calc_emim(a, b):
  n_a = n_docs_word(a)
  n_b = n_docs_word(b)
  n_ab = n_docs_both(a, b)
  n = n_docs()
  exp = n * (n_ab / (n_a * n_b))
  emim = n_ab * np.log(exp) if exp != 0 else 0
  return (emim, b)
  
def calc_x(a, b):
  n_a = n_docs_word(a)
  n_b = n_docs_word(b)
  n_ab = n_docs_both(a, b)
  n = n_docs()
  x = (n_ab - (1 / n) * n_a * n_b) ** 2 / (n_a * n_b)
  return (x,b)
  
def calc_dice(a, b):
  n_a = n_docs_word(a)
  n_b = n_docs_word(b)
  n_ab = n_docs_both(a, b)
  dice = n_ab / (n_a + n_b)
  return (dice,b)

In [0]:
def create_metrics_table(query):
  top_10_table = pd.DataFrame()
  MIM = []
  EMIM = []
  X = []
  Dice = []
  
  for word in ranking:
    if word != query:
      bisect.insort(MIM, calc_mim(query,word))
      bisect.insort(EMIM,calc_emim(query, word))
      bisect.insort(X,calc_x(query, word))
      bisect.insort(Dice,calc_dice(query, word))

  MIM.reverse()
  EMIM.reverse()
  X.reverse()
  Dice.reverse()
  
  top_10_table['MIM'] = [word[1] for word in MIM]
  top_10_table['EMIM'] = [word[1] for word in EMIM]
  top_10_table['X²'] = [word[1] for word in X]
  top_10_table['Dice'] = [word[1] for word in Dice]
  
  return top_10_table.head(10)

In [0]:
table_1 = create_metrics_table(queries[0])
table_2 = create_metrics_table(queries[1])
table_3 = create_metrics_table(queries[2])
table_4 = create_metrics_table(queries[3])
table_5 = create_metrics_table(queries[4])

####Tabela para a consulta: 'Bolsonaro'

In [200]:
table_1.index += 1
table_1

,MIM,EMIM,X²,Dice
1,única,jair,jair,jair
2,últimas,presidente,psl,presidente
3,ódio,anos,anos,brasil
4,íntegra,psl,visita,anos
5,índia,brasil,então,governo
6,ídolo,visita,israel,segundo
7,áreas,então,seguida,afirmou
8,árabe,israel,críticas,disse
9,água,governo,campanha,ser
10,zilhões,afirmou,fazer,sobre


####Tabela para a consulta: 'Educação'

In [201]:
table_2.index += 1
table_2

,MIM,EMIM,X²,Dice
1,ávila,vélez,vélez,vélez
2,zero,sabemos,sabemos,sabemos
3,vélez,ricardo,ricardo,ricardo
4,visando,afirmação,afirmação,afirmação
5,vinicius,pasta,soberana,pasta
6,vinculando,completou,saúde,completou
7,vigor,apenas,rodriguez,soberana
8,vidas,seguida,risca,saúde
9,viciado,porque,didáticos,rodriguez
10,verídica,soberana,constitucional,risca


####Tabela para a consulta: 'Ministério'

In [202]:
table_3.index += 1
table_3

,MIM,EMIM,X²,Dice
1,ínsita,público,tcu,público
2,zero,tcu,público,diz
3,you,militares,procuradores,militares
4,wti,voz,petrobrás,segundo
5,wikileaks,união,força,sobre
6,whatsapp,diz,conselho,voz
7,web,deste,aplicação,união
8,vítima,tribunal,voz,tribunal
9,véspera,procuradores,união,ainda
10,válidos,petrobrás,militares,deste


####Tabela para a consulta: 'Brasil'

In [203]:
table_4.index += 1
table_4

,MIM,EMIM,X²,Dice
1,útil,brasileira,brasileira,bolsonaro
2,única,outros,partir,jair
3,índia,bolsonaro,onde,sobre
4,ávila,jair,resistência,brasileira
5,áreas,onde,luiz,segundo
6,áquila,países,outros,ser
7,zonas,estados,tempo,país
8,zilhões,partir,momento,governo
9,zero,tempo,países,ter
10,you,momento,regime,ainda


####Tabela para a consulta: 'Economia'

In [204]:
table_5.index += 1
table_5

,MIM,EMIM,X²,Dice
1,única,guedes,guedes,guedes
2,ávila,proposta,proposta,previdência
3,zero,previdência,previdência,proposta
4,zeca,reforma,cenário,reforma
5,wti,bilhões,reforma,paulo
6,walter,paulo,últimos,ano
7,véspera,últimos,meses,bilhões
8,votada,meses,maiores,ministro
9,volatilidade,maiores,empresas,nesta
10,vocação,empresas,audiência,país


### 2. De acordo com a métrica que deu os melhores resultados na sua opinião, execute agora cada consulta (usando a abordagem documento- ou termo-por-vez)  expandido-a com: os top-3, top-5 e top-10 documentos. O que acontece com a precisão dos resultados em cada caso? Aumenta ou diminui? Justifique bem sua resposta.